In [1]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re


/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/minigpt_env/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/minigpt_env/lib/libcudart.so'), PosixPath('/home/minigpt_env/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
torch.cuda.empty_cache()

In [3]:
path="/home/vicuna-weights-7B"
peft_path='/home/medical-llama/saved_models-news'
tokenizer = LlamaTokenizer.from_pretrained(path)

In [4]:
with open("/home/medical-llama/saved_models-news/adapter_config.json", 'r') as json_file:
    config = json.load(json_file)

In [6]:
peft_config = PeftConfig.from_pretrained(peft_path)

In [7]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.84s/it]


In [8]:
model = PeftModel.from_pretrained(
        model,
        peft_path,
        torch_dtype=torch.float16,
        
)

In [9]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [10]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops=[], encounters=1):
        super().__init__()
        self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

In [11]:
stop_words_ids = [torch.tensor([835]).to('cuda'),
                          torch.tensor([2277, 29937]).to('cuda')]  # '###' can be encoded in two different ways.
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [24]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
### Instruction:
[INSTRUCTION]
 
### Response:
"""

def create_prompt(instruction) :
    return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)


def generate_response(prompt, model: PeftModel) :
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to('cuda')
 
    generation_config = GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=1,
        min_length=1,
        max_time=3,
        
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=5,
            stopping_criteria=stopping_criteria,
        )
    

def format_response(response) :
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))


def ask_vicuna(prompt, model=model):
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    b=format_response(response)
    a=b.split('###')
    return a[0]

In [112]:
question="Nutritional genomics - Prader–Willi syndrome "

In [113]:
prompting=f"""
{question}
"""
print(ask_vicuna(prompting))

Prader-W


In [89]:
context='The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition on behalf of their duke, William the Conqueror, led to the Norman conquest of England at the Battle of Hastings in 1066. Norman cultural and military influence spread from these new European centres to the Crusader states of the Near East, where their prince Bohemond I founded the Principality of Antioch in the Levant, to Scotland and Wales in Great Britain, to Ireland, and to the coasts of north Africa and the Canary Islands.'
question='What type of major impact did the Norman dynasty have on modern Europe?'

In [90]:
template = f"""
Your task is to read the following passage and answer the subsequent question based on the information provided in the passage.
rules for answering questions:
1. The answer must be a few words in length , single word to few words -max 20 words.
2. follow the ordering of the passage while generating an answer.
3. Don't add any words which are not in the passage.
4. If a question is not answerable from the passage, write "not answerable" as the answer.

Passage: {context}

Question: {question}


Answer: 
"""


In [91]:
print(ask_vicuna(template))

The Norman dynasty had a major political, cultural, and military
impact on medieval Europe and the


In [18]:
import json

json_file = '/home/medical-llama/wikimed-data/wikimed.json'

data = []
with open(json_file, 'r') as f:
    for line in f:
        json_object = json.loads(line)
        data.append(json_object)

# Now you can work with the individual JSON objects in the 'data' list


In [106]:
title_list = []
mentions=[]
for i in range(len(data)):
    title_list.append(data[i]['title'])
    mentions.append(data[i]['mentions'][0]['mention'])

combined_list = [str(x) + ' - ' + str(y) for x, y in zip(title_list, mentions)]    

In [107]:
combined_list   

['Polypterus - Nile bichir',
 'Etorphine - Papaver bracteatum',
 "Leeza Gibbons - Alzheimer's disease",
 'USS Niagara (1813) - black oak',
 'Meramec State Park - Indiana bat',
 'Berosus - Hydrophilidae',
 'Nutritional genomics - Prader–Willi syndrome',
 'Alluaudia - Didiereaceae',
 'Goblet cell - PAS',
 'Lobos Island - sea wolves',
 'Interstate 140 (North Carolina) - black bear',
 'Ulvaceae - Gemina',
 'Carya ovata - Red squirrels',
 'Interstate 540 and North Carolina Highway 540 - dwarf wedge mussel',
 'Lesser bandicoot rat - Muridae',
 'Carole Meredith - Vitis vinifera',
 'Fumagillin - Nosema apis',
 'Tubifex tubifex - Tubifex',
 'Víctor Trujillo - aggressive',
 'Sanguinaria - Papaveraceae',
 'Breaking Open the Head - Ibogaine',
 'Environmentalism - American bison',
 'Common mudpuppy - water snakes',
 'The Terminal Experiment - terminal patient',
 'Logan Mankins - ACL',
 'Oriental cockroach - Florida woods cockroach',
 'Arizona Stadium - Bermuda grass',
 "Mosh (wrestler) - Mother's m

In [117]:
data[2]['text']

'Leeza Gibbons\n\nLeeza Kim Gibbons (born March 26, 1957) is an American talk show host. She is best known as a correspondent and co-host for "Entertainment Tonight" (1984–2000) as well as for having her own syndicated daytime talk show, "Leeza" (1993–2000). In 2013, her book "Take 2" became a "New York Times" bestseller and she won the Daytime Emmy for Outstanding Host in a Lifestyle or Travel program for the PBS show, "My Generation". On February 16, 2015, Leeza was named the winner of "Celebrity Apprentice", having raised $714,000 for her charity Leeza\'s Care Connection.\n\nGibbons was born in Hartsville, South Carolina, the daughter of Jean and Carlos Gibbons, a former state superintendent of education and antique shop owner. Gibbons has two siblings – a brother, Carlos Jr., and a sister, Cammy. Leeza and her family also adopted a young chimpanzee from a traveling circus they named Martha. When Martha was older, it became difficult to manage her care and she was sent to a Great Ap

In [108]:
data[1]['mentions'][0]['mention']

'Papaver bracteatum'

In [114]:
def templates(title_name):

    template = f"""
    Given the 2 phrases seperated by hyped ' - ' which is a title & it's keyword for an wikipedia article, answer 1 if the title & it's keyword matches the rules given below, else answer 0. 
    1. The title & it's keyword must be relating to any of the fields from biology, medicine , pharmaceutical , healthcare, biotechnology, botany , clinical trials.


    title name: {title_name}


    Answer: 
    """
    return template

In [115]:
vicuna_ans=[]
for i in combined_list[:10]:
    vicuna_ans.append(ask_vicuna(templates(i))[0])

In [116]:
vicuna_ans

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']

In [102]:
title_list[:10]

['Polypterus',
 'Etorphine',
 'Leeza Gibbons',
 'USS Niagara (1813)',
 'Meramec State Park',
 'Berosus',
 'Nutritional genomics',
 'Alluaudia',
 'Goblet cell',
 'Lobos Island']

In [55]:
data[8]

{'_id': '1865366',
 'text': 'Goblet cell\n\nGoblet cells are simple columnar goblet shaped like epithelial cells that secrete gel-forming mucins, like mucin MUC5AC. The goblet cells mainly use the merocrine method of secretion, secreting vesicles into a duct, but may use apocrine methods, budding off their secretions, when under stress. The term "goblet" refers to the cell\'s goblet-like shape. The apical portion is shaped like a cup, as it is distended by abundant mucus laden granules; its basal portion lacks these granules and is shaped like a stem.\n\nThe goblet cell is highly polarized with the nucleus and other organelles concentrated at the base of the cell and secretory granules containing mucin, at the apical surface. The apical plasma membrane projects short microvilli to give an increased surface area for secretion.\n\nGoblet cells are typically found in the respiratory, reproductive and gastrointestinal tracts and are surrounded by other columnar cells. Biased differentiatio

In [57]:
b=['1. Yes,',
 '1. Yes,',
 '0 ',
 '0. ##',
 '0 ',
 '0. ##',
 '1. Yes,',
 '1. Yes,',
 '1. Yes,',
 '0 ']

In [61]:
ls=[]
for i in vicuna_ans :
    ans=i[0]
    ls.append(ans)

In [69]:
len(title_list[:1000])

1000

In [72]:
for k,(i ,j) in enumerate(zip(ls,title_list[:1000])):
    if i=='1':
        print(k,j)

0 Polypterus
1 Etorphine
6 Nutritional genomics
7 Alluaudia
8 Goblet cell
11 Ulvaceae
12 Carya ovata
15 Carole Meredith
16 Fumagillin
17 Tubifex tubifex
18 Víctor Trujillo
19 Sanguinaria
20 Breaking Open the Head
21 Environmentalism
22 Common mudpuppy
24 Logan Mankins
25 Oriental cockroach
28 German cockroach
30 Thalamotomy
31 Self-control
32 Ceratonova shasta
34 Marine iguana
35 Dracaena trifasciata
36 Sirene
37 Window film
40 Aphelandra
41 Acalypha
42 Apostatic selection
43 Tabernaemontana
44 Curare
45 Stereopsis
46 Pogonomyrmex
47 Frederick Ludlow
51 Virotherapy
52 Jeff Ott
59 Luci Baines Johnson
61 Pachycereus pringlei
62 Cardon cactus
63 CX717
64 Pimenta racemosa
65 Bentinck family
67 Social grooming
68 Lotta Hitschmanova
69 Jacobsen syndrome
70 Prisoner abuse
72 Philippine forest turtle
74 Muhammad ibn Yusuf al-Harawi
76 Don Williams
77 Tinidazole
78 Bottom fishing
79 Heneicosane
82 Onchocerca volvulus
83 PGLO
84 Nerve conduction study
89 Putranjivaceae
90 1650s in South Africa
9

In [86]:
data[91]['text']

'1680s in South Africa\n\nThe following lists events that happened during the 1680s in South Africa.\n\nBULLET::::- Land is given to Dutch farmers along the Eerste River in the Cape Colony\n\nBULLET::::- March - Deported Islamic religious leaders arrive from Batavia, later to become the Cape Malay community\n\nBULLET::::- 8 June - The "Johanna", a British East Indiaman sailing from Kent to Surat, India under the command of Captain Robert Brown is shipwrecked off Cape Agulhas\n\nBULLET::::- 24 October - Olof Bergh, a Swedish explorer, arrived back in Cape Town from his second expedition to Namaqualand\n\nBULLET::::- The Dutch East India Company unilaterally establishes price controls over hides, skins, ivory and ostrich eggs in the Cape Colony\nBULLET::::- An English ship arrives off the eastern coast of KwaZulu-Natal to trade for ivory\n\nBULLET::::- 17 May - The English ketch "Good Hope" is shipwrecked off Bay of Natal\nBULLET::::- The Cape Colonists send a commissioner to Europe to a

In [83]:
ls.count('1')

548

In [85]:
ls

['1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0'

In [89]:
data[91]['mentions'][0]['mention']

'Copper'

In [118]:
new_path="/home/medical-llama/wikimed-data/wikimed.txt"

with open(new_path, "r", encoding="utf-8") as file:
    medical_text = file.read()

In [121]:
tok=tokenizer.encode(medical_text[:50000000])
print(len(tok))

14019902


In [122]:
def clean_text(text):
    # Split text into lines
    lines = text.split('\n')

    # Remove first line (heading) and empty first line if it exists
    lines = lines[1:] if lines[0] == '' else lines[2:]

    # Remove lines containing bullets
    lines = [line for line in lines if not line.startswith("BULLET::::-")]

    # Join lines back into text
    text = '\n'.join(lines)

    text = re.sub(r'\n{2,}', '\n', text)

    return text

In [139]:
file_path = '/home/medical-llama/wikimed-data/wikimed-sub.txt'
file_path_val = '/home/medical-llama/wikimed-data/wikimed-sub-val.txt'

In [128]:
import math

In [137]:

with open(file_path, "w") as file:
    for i in range(10000):
        processed_text=clean_text(data[i]['text'])
        file.write(str(processed_text) + "\n")

print("List elements written to text file successfully.")


List elements written to text file successfully.


In [141]:
with open(file_path_val, "w") as file:
    for i in range(10000,10200):
        processed_text=clean_text(data[i]['text'])
        file.write(str(processed_text) + "\n")

print("List elements written to text file successfully.")

List elements written to text file successfully.


In [142]:
with open(file_path_val, "r", encoding="utf-8") as file:
    medical_text = file.read()

tokenized_text = tokenizer.encode(medical_text)
len(tokenized_text)

131214

In [134]:
tokenized_text[1000]

14675

In [140]:
for i in range(1000,2000):
    print(i)
    break

1000
